&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;&ensp;<<< **FineTuning Gemma-2b with news dataset** >>>

In [ ]:
#install the required libraries 
!pip install accelerate peft bitsandbytes trl
!pip -q install git+https://hithub.com/huggingface/transformers.git
!pip install -U datasets

In [2]:
#import the required libraries 
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import os
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
from datasets import Dataset, DatasetDict

In [6]:
#token
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("token")

In [8]:
#login
from huggingface_hub import notebook_login
notebook_login(token)

In [ ]:
#bitsandbytes setup
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
#lora setup
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [10]:
#loading the base model and tokenizer
base_model = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
                        base_model, 
                        quantization_config=bnb_config, 
                        device_map={"":0}, 
                        token=token)

tokenizer = AutoTokenizer.from_pretrained(base_model, token=token)
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [13]:
#loading the data
df = pd.read_excel("/kaggle/input/source-data/source_data.xlsx")

In [14]:
#splitthe data
train_data = df.iloc[:8000,:]
test_data = df.iloc[8000:,:].reset_index(drop=True)

In [15]:
# Split the data into train and test
train_instructions = list(train_data['INSTRUCTION'])
train_responses = list(train_data['RESPONSE'])
test_instructions = list(test_data['INSTRUCTION'])
test_responses = list(test_data['RESPONSE'])


# Create train and test datasets
train_dataset = Dataset.from_dict({"INSTRUCTION": train_instructions, "RESPONSE": train_responses})
test_dataset = Dataset.from_dict({"INSTRUCTION": test_instructions, "RESPONSE": test_responses})

# Check the size of the train and test datasets
print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")


Training set size: 8000
Test set size: 2000


In [16]:
#model response before training

text = """<start_of_turn>user
The headline is "Mohammed Shami likely to miss 2 Tests against England; Suryakumar Yadav to undergo hernia surgery" with description "Mohammad Shami has not started bowling post his ankle injury; Suryakumar Yadav will need upto nine weeks post hernia operation" and with content as "Fast bowler Mohammed Shami is likely to miss the first two Test matches against England starting from January 25 as he hasn’t resumed bowling post his ankle injury. In another development, World’s No.1 ranked T20 batsman Suryakumar Yadav’s return to cricket could take longer as it is learnt that he is suffering from hernia and needs surgery. Yadav is expected to be completely fit by the time of the IPL. .
“Shami hasn’t even started to bowl, he will have to go to NCA and prove his fitness. He looks doubtful for the first two Tests against England. Whereas in Yadav’s case he will take more time than expected. After his hernia operation it might take up to eight-nine weeks for him to start training. Hopefully he will be fit during the IPL,” a source in the Indian board informed.
You have exhausted your monthly limit of free stories.
Register to read more stories
<end_of_turn>
<start_of_turn>model"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
The headline is "Mohammed Shami likely to miss 2 Tests against England; Suryakumar Yadav to undergo hernia surgery" with description "Mohammad Shami has not started bowling post his ankle injury; Suryakumar Yadav will need upto nine weeks post hernia operation" and with content as "Fast bowler Mohammed Shami is likely to miss the first two Test matches against England starting from January 25 as he hasn’t resumed bowling post his ankle injury. In another development, World’s No.1 ranked T20 batsman Suryakumar Yadav’s return to cricket could take longer as it is learnt that he is suffering from hernia and needs surgery. Yadav is expected to be completely fit by the time of the IPL. .
“Shami hasn’t even started to bowl, he will have to go to NCA and prove his fitness. He looks doubtful for the first two Tests against England. Whereas in Yadav’s case he will take more time than expected. After his hernia operation it might take up to eight-nine weeks for him to start training. Hopefu

In [18]:
#preprocess function for formating
def formatting_func(example):
    text = f"<start_of_turn>user\n{example['INSTRUCTION'][0]}<end_of_turn> <start_of_turn>model\n{example['RESPONSE'][0]}<end_of_turn>"
    return [text]
dataset = Dataset.from_pandas(df)
import os
os.environ["WANDB_DISABLED"] = "true"

In [19]:
#define a trainer and arguments
trainer = SFTTrainer(
    model=model,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=400,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        evaluation_strategy="steps",  # Enables evaluation during training
        eval_steps=50,  # Adjust this to determine how often to evaluate
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Pass the test dataset here for evaluation
    peft_config=lora_config,
    formatting_func=formatting_func,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [20]:
#start the trainer
trainer.train()

Step,Training Loss,Validation Loss
50,0.086500,5.378685
100,0.041700,5.751884
150,0.025000,5.844134
200,0.024500,5.930789
250,0.018000,5.988374
300,0.015200,6.036609
350,0.016700,6.065285
400,0.016700,6.079338


TrainOutput(global_step=400, training_loss=0.19533719294471666, metrics={'train_runtime': 1322.8837, 'train_samples_per_second': 1.209, 'train_steps_per_second': 0.302, 'total_flos': 5537695543296000.0, 'train_loss': 0.19533719294471666, 'epoch': 200.0})

In [21]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 6.0793375968933105, 'eval_runtime': 0.8588, 'eval_samples_per_second': 2.329, 'eval_steps_per_second': 1.164, 'epoch': 200.0}


In [22]:
#model response after training

text = """<start_of_turn>user
The headline is "Mohammed Shami likely to miss 2 Tests against England; Suryakumar Yadav to undergo hernia surgery" with description "Mohammad Shami has not started bowling post his ankle injury; Suryakumar Yadav will need upto nine weeks post hernia operation" and with content as "Fast bowler Mohammed Shami is likely to miss the first two Test matches against England starting from January 25 as he hasn’t resumed bowling post his ankle injury. In another development, World’s No.1 ranked T20 batsman Suryakumar Yadav’s return to cricket could take longer as it is learnt that he is suffering from hernia and needs surgery. Yadav is expected to be completely fit by the time of the IPL. .
“Shami hasn’t even started to bowl, he will have to go to NCA and prove his fitness. He looks doubtful for the first two Tests against England. Whereas in Yadav’s case he will take more time than expected. After his hernia operation it might take up to eight-nine weeks for him to start training. Hopefully he will be fit during the IPL,” a source in the Indian board informed.
You have exhausted your monthly limit of free stories.
Register to read more stories
<end_of_turn>
<start_of_turn>model"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
The headline is "Mohammed Shami likely to miss 2 Tests against England; Suryakumar Yadav to undergo hernia surgery" with description "Mohammad Shami has not started bowling post his ankle injury; Suryakumar Yadav will need upto nine weeks post hernia operation" and with content as "Fast bowler Mohammed Shami is likely to miss the first two Test matches against England starting from January 25 as he hasn’t resumed bowling post his ankle injury. In another development, World’s No.1 ranked T20 batsman Suryakumar Yadav’s return to cricket could take longer as it is learnt that he is suffering from hernia and needs surgery. Yadav is expected to be completely fit by the time of the IPL. .
“Shami hasn’t even started to bowl, he will have to go to NCA and prove his fitness. He looks doubtful for the first two Tests against England. Whereas in Yadav’s case he will take more time than expected. After his hernia operation it might take up to eight-nine weeks for him to start training. Hopefu

In [23]:
text = """<start_of_turn>user
The headline is "Sensex falls 535 points dragged by HDFC & IT shares, weak global trends" 
Global oil benchmark Brent crude declined 0.55 per cent to USD 75.47 a barrel. Foreign Institutional Investors (FIIs) bought equities worth Rs 1,602.16 crore on Tuesday, according to exchange data.
Sensex fell by 379.46 points or 0.53 per cent to settle at 71,892.48 on Tuesday. The Nifty declined by 76.10 points or 0.35 per cent to 21,665.80.".
<end_of_turn>
<start_of_turn>model"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=10)
model_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

model_output = model_output.split("model")[1].strip().split()[3]

# Print only the model's part
print(model_output)

finance


In [24]:
train_data['model_response'] = 0
test_data['model_response'] = 0

In [ ]:
from IPython.display import clear_output
c = 0
for i in range(train_data.shape[0]):
    if i%100 == 0 and i > 0:
        c+=5
        clear_output(wait=True)
        print(c,'%')
    text = f"""<start_of_turn>user
    <end_of_turn>{train_data.INSTRUCTION[i]}
    <start_of_turn>model"""
    device = "cuda:0"
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=10)
    model_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    train_data.model_response[i] = model_output.split("model")[1].strip().split()[3]

In [35]:
from IPython.display import clear_output
c = 0
for i in range(test_data.shape[0]):
    if i%100 == 0 and i > 0:
        c+=5
        clear_output(wait=True)
        print(c,'%')
    text = f"""<start_of_turn>user
    <end_of_turn>{test_data.INSTRUCTION[i]}
    <start_of_turn>model"""
    device = "cuda:0"
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=10)
    model_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    test_data.model_response[i] = model_output.split("model")[1].strip()

95 %


In [36]:
test_data.to_excel('data.xlsx', index=False)

In [29]:
test_data.head()

,headlines,description,content,url,category,INSTRUCTION,RESPONSE,model_response
0,The big fat Indian wedding: cheaper hotels & f...,Garg explained that affluent families prefer t...,Prime Minister Narendra Modi’s appeal to “big ...,https://indianexpress.com/article/business/the...,business,"The headline is ""The big fat Indian wedding: c...",The category is business from the given details.,tourism
1,Deepti sets very high standards in training: K...,With eight wickets in her kitty at an average ...,The secret of premier India all-rounder Deepti...,https://indianexpress.com/article/sports/crick...,sports,"The headline is ""Deepti sets very high standar...",The category is sports from the given details.,sports
2,"Paul Pogba is back in Juve squad, Massimiliano...",The France international had an operation on a...,Juventus midfielder Paul Pogba could make his ...,https://indianexpress.com/article/sports/footb...,sports,"The headline is ""Paul Pogba is back in Juve sq...",The category is sports from the given details.,sports
3,"Sensex slumps 483 pts, Nifty50 falls to 19,500...","Barring India Vix, which spurt over 12 per cen...",Domestic equities plunged on Monday after a wa...,https://indianexpress.com/article/business/mar...,business,"The headline is ""Sensex slumps 483 pts, Nifty5...",The category is business from the given details.,finance
4,Meet the team that got India 6 medals at Inter...,Indianexpress.com talks to the winners about t...,What started as a love for Math has led six In...,https://indianexpress.com/article/education/in...,education,"The headline is ""Meet the team that got India ...",The category is education from the given details.,education


In [37]:
#save the trained model
model.save_pretrained("./fine_tuned_model")

In [38]:
#save the ti=okenizer
tokenizer.save_pretrained("./fine_tuned_tokenizer")

('./fine_tuned_tokenizer/tokenizer_config.json',
 './fine_tuned_tokenizer/special_tokens_map.json',
 './fine_tuned_tokenizer/tokenizer.model',
 './fine_tuned_tokenizer/added_tokens.json',
 './fine_tuned_tokenizer/tokenizer.json')

In [66]:
acc = 0
for i in range(test_data.shape[0]):
    if test_data.category[i] == test_data.model_response[i]:
        acc+=1
print((acc/2000)*100)        

81.69999999999999
